In [87]:
! pip install -U -q gensim

In [1]:
import pandas as pd
import os


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
path = './oss_bigquery_exports/readmes'
files = list(os.walk(path))[0][2]

In [3]:
df = pd.read_csv('{}/{}'.format(path, files[0]))

In [4]:
import re
from sklearn.feature_extraction.text import strip_accents_ascii, strip_tags

headlines = re.compile(r"#+\s*[^\n]+\n")
md_links = re.compile('\[[^\]]+\]\([^\)]+\)')
sentance = re.compile(r"\.\s+")
links = re.compile(r"https?://[^\s]+")
code_ticks = re.compile(r"``?`?[^`]+``?`?")
token_pattern = re.compile(r"(?u)\b\w\w+\b")
tokenizer = lambda doc: token_pattern.findall(doc)
underscore = re.compile(r"\w*_\w*")
space = re.compile(r'\s+')


def preprocessor(s):

    # Capitalization won't help us
    s = s.lower()

    # Remove code and markdown headlines
    s = re.sub(code_ticks, '', s)    
    s = re.sub(headlines, '', s)
    s = re.sub(md_links, '', s)
    s = re.sub(links, '', s)

    # ASCII our text and remove html tags
    s = strip_accents_ascii(s)
    s = strip_tags(s)

    # Underscores imply variable names, which are
    # never useful. Get rid of anything in camelcase? 
    s = re.sub(underscore, '', s)


    # Split on sentances, tokenize within the sentance, then replace 
    # sentance with \t separator for starspace/fasttext
    s = [i for i in sentance.split(s)]
    s = [' '.join(tokenizer(i)) for i in s]
    s = '\t'.join([i for i in s if i])

    # Get rid of useless little readmes
    if len(space.split(s)) < 4:
        return None

    return s

In [5]:
def process_df(df):
    df = df[df.content.notna()].reset_index()
    df['content'] = df.content.map(preprocessor)
    df = df[df.content.notna()].reset_index()
    df['repo_owner'] = df.repo_name.map(lambda s: s.split('/')[0])
    return df

In [6]:
dfs = (pd.read_csv('{}/{}'.format(path, f)) for f in files)

In [ ]:
with open('file_lookup.csv', 'w') as f:
    for df in dfs:
        df = process_df(df)
        for i,c in zip(df.id, df.content):
            f.write('{},{}\n'.format(i,c))

In [101]:
by_owner = df.groupby('repo_owner').apply(lambda df: ' \t '.join(df.content))

In [102]:
with open('by_owner.txt', 'w') as f:
    for c in by_owner:
        f.write(c + '\n')